Ensembling different models

In [1]:
from google.cloud import storage
from io import BytesIO

client = storage.Client()
storage_client = storage.Client(project = 'irkml1')
bucket = storage_client.get_bucket("aindstorm_bucket")
blob1 = storage.blob.Blob("train_3lags_semibalanced.csv",bucket)
content1 = blob1.download_as_string()

In [25]:
blob2 = storage.blob.Blob("train_3lags_v1.csv",bucket)
content2 = blob2.download_as_string()

In [2]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random
import os
import scipy.stats as ss
import scipy
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier, Pool, sum_models, to_classifier
from sklearn.model_selection import KFold
from sklearn.utils.class_weight import compute_class_weight


In [3]:
df2 = pd.read_csv(BytesIO(content1), low_memory=False)

In [26]:
df = pd.read_csv(BytesIO(content2), low_memory=False)

In [4]:
df2 = df2.sample(frac=1).reset_index(drop=True)

In [6]:
df2.columns

Index(['requester', 'service', 'service_title', 'mfc', 'internal_status',
       'external_status', 'order_type', 'department_id', 'custom_service_id',
       'service_level', 'is_subdep', 'is_csid', 'proc_time', 'dayofweek',
       'day_part', 'month', 'week', 'year', 'person', 'sole', 'legal',
       'auto_ping_queue', 'win_count', 'service_1', 'service_title_1', 'mfc_1',
       'internal_status_1', 'external_status_1', 'order_type_1',
       'department_id_1', 'custom_service_id_1', 'service_level_1',
       'is_subdep_1', 'is_csid_1', 'proc_time_1', 'dayofweek_1', 'day_part_1',
       'month_1', 'week_1', 'year_1', 'person_1', 'sole_1', 'legal_1',
       'auto_ping_queue_1', 'win_count_1', 'service_2', 'service_title_2',
       'mfc_2', 'internal_status_2', 'external_status_2', 'order_type_2',
       'department_id_2', 'custom_service_id_2', 'service_level_2',
       'is_subdep_2', 'is_csid_2', 'proc_time_2', 'dayofweek_2', 'day_part_2',
       'month_2', 'week_2', 'year_2', 'perso

In [5]:
df2 = df2[['service_title', 'service_title_1', 'requester_type', 'gender', 'age']]

In [57]:
df2.dropna(inplace=True)

In [64]:
y = df2['service_title']
X = df2.drop(['service_title'], axis=1)

In [72]:
le1 = preprocessing.LabelEncoder()
le2 = preprocessing.LabelEncoder()
le3 = preprocessing.LabelEncoder()
le4 = preprocessing.LabelEncoder()

In [73]:
y = le1.fit_transform(y)
X['service_title_1'] = le2.fit_transform(X['service_title_1'])
X['requester_type'] = le3.fit_transform(X['requester_type'])
X['gender'] = le4.fit_transform(X['gender'])

In [67]:
le = preprocessing.LabelEncoder()
for col in categorical:
    X[col] = le.fit_transform(X[col])

In [73]:
y = le1.fit_transform(y)

In [71]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [75]:
clf = RandomForestClassifier(n_estimators=100)

In [ ]:
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
y_pred  = clf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

In [100]:
sab = pd.read_csv("sample_submission_ld.csv")

sab_pred = sab[['requester']]

sab_pred = pd.merge(sab_pred, df, how='left', on='requester')

sab_pred.drop(['service_3',
'service_title_3',
'mfc_3',
'internal_status_3',
'external_status_3',
'order_type_3',
'department_id_3',
'custom_service_id_3',
'service_level_3',
'is_subdep_3',
'is_csid_3',
'proc_time_3',
'dayofweek_3',
'day_part_3',
'person_3',
'sole_3',
'legal_3',
'auto_ping_queue_3',
'win_count_3', 'year_3', 'month_3', 'week_3'], axis=1 , inplace=True)

sab_pred.drop(['requester'], axis=1, inplace=True)

In [65]:
sab_pred = sab_pred[['service_title', 'requester_type', 'gender', 'age']]

In [66]:
sab_pred.fillna(sab_pred.mode().iloc[0], inplace=True)

In [67]:
sab_pred[['service_title', 'requester_type', 'gender']] = sab_pred[['service_title', 'requester_type', 'gender']].astype('int64')

In [84]:
sab_pred['service_title'].loc[sab_pred['service_title'] == 151] = 98
sab_pred['service_title'].loc[sab_pred['service_title'] == 408] = 98
sab_pred['service_title'].loc[sab_pred['service_title'] == 509] = 98
sab_pred['service_title'].loc[sab_pred['service_title'] == 945] = 98

In [86]:
sab_pred['service_title_1'] = le2.transform(sab_pred['service_title'])
sab_pred['requester_type'] = le3.transform(sab_pred['requester_type'])
sab_pred['gender'] = le4.transform(sab_pred['gender'])

In [88]:
sab_pred.drop(['service_title'], axis=1, inplace=True)

In [89]:
sab_pred = sab_pred[['service_title_1', 'requester_type', 'gender', 'age']]

In [90]:
sab_pred

,service_title_1,requester_type,gender,age
0,993,0,1,34.0
1,48,0,0,29.0
2,110,0,0,61.0
3,742,0,0,31.0
4,493,0,1,37.0
...,...,...,...,...
44439,168,0,0,32.0
44440,82,0,1,53.0
44441,886,0,1,34.0
44442,404,0,1,31.0


In [91]:
y_pred  = clf.predict(sab_pred)

In [92]:
sab['service_title'] = le1.inverse_transform(y_pred)

In [96]:
sab.to_csv('rf_sub_0.csv', index=False)

In [97]:
sab_nn = pd.read_csv('nn_5fold_2models_0.csv')

In [98]:
sab_xgb = pd.read_csv('xgb_sub_0.csv')

In [99]:
sab_rf = pd.read_csv('rf_sub_0.csv')

In [103]:
idx = sab_pred.loc[sab_pred['order_count'] == 1].index

In [105]:
sab['service_title'] = sab_nn['service_title']

In [121]:
sab_all = sab[:]

In [123]:
sab_all['nn'] = sab_nn['service_title']
sab_all['xgb'] = sab_xgb['service_title']
sab_all['rf'] = sab_rf['service_title']

/home/jupyter/.pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/jupyter/.pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jupyter/.pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [118]:
def voting(x):
    d = {}
    x1 = x['nn']
    x2 = x['xgb']
    x3 = x['rf']
    if x2 == x3:
        return x2
    return x1

In [125]:
sab_all['service_title'].iloc[idx] = sab_all.iloc[idx].apply(voting, axis=1)

/home/jupyter/.pyenv/versions/3.7.4/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [128]:
sab_all = sab_all[['requester', 'service_title']]

In [130]:
sab_all['service_title'].value_counts(normalize=True)

603     0.127059
134     0.107416
98      0.071978
907     0.067478
207     0.064103
          ...   
514     0.000023
965     0.000023
1       0.000023
274     0.000023
1105    0.000023
Name: service_title, Length: 138, dtype: float64

In [131]:
sab_nn['service_title'].value_counts(normalize=True)

134     0.107056
603     0.099631
98      0.073238
207     0.069818
907     0.064733
          ...   
514     0.000023
965     0.000023
274     0.000023
1       0.000023
1105    0.000023
Name: service_title, Length: 138, dtype: float64

In [132]:
sab_all.to_csv('vot_sub_0.csv', index=False)